In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.functions import (
    col, collect_set, concat, count, current_timestamp, explode,
    lit, lower, size, split, udf
)
from pyspark.sql.types import (
    ArrayType, BooleanType, StringType,
    StructField, StructType, IntegerType
)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [3]:
# Get spark context
sc = SparkContext.getOrCreate(conf=conf) # conf is provided by sparkmonitor
spark = SparkSession(sc)

## Orange Book Data (NDA, Patents)

This will work with the `./data/Orange_Book/nber_1985_2016/FDA_drug_patents.csv` file in this repo.

In [4]:
orange_book_df = spark.read.csv('./data/Orange_Book/nber_1985_2016/FDA_drug_patents.csv', header=True, inferSchema=True)
# Explore Columns
orange_book_df.printSchema()
orange_book_df.show()

root
 |-- _c0: integer (nullable = true)
 |-- edition: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- application_number: integer (nullable = true)
 |-- product_number: integer (nullable = true)
 |-- patent_number: string (nullable = true)
 |-- patent_expiration: timestamp (nullable = true)
 |-- use_code: string (nullable = true)
 |-- active_ingredient: string (nullable = true)
 |-- trade_name: string (nullable = true)
 |-- DS: string (nullable = true)
 |-- DP: string (nullable = true)
 |-- ped_extension: double (nullable = true)
 |-- delist_requested: string (nullable = true)

+---+-------+----------------+------------------+--------------+-------------+-------------------+--------+-----------------+----------+---+---+-------------+----------------+
|_c0|edition|application_type|application_number|product_number|patent_number|  patent_expiration|use_code|active_ingredient|trade_name| DS| DP|ped_extension|delist_requested|
+---+-------+----------------+-

In [5]:
sample_df = (orange_book_df
                .filter(orange_book_df['active_ingredient'].isin('MORPHINE SULFATE'))
                .select('patent_number', 'application_number'))

In [6]:
appln_numbers = sample_df.select('application_number').distinct()
appln_numbers.show()

+------------------+
|application_number|
+------------------+
|            204223|
|             21260|
|             21671|
|             20616|
+------------------+



In [7]:
appln_numbers_list = list(map(lambda x: x.application_number, appln_numbers.collect()))
appln_numbers_list

[204223, 21260, 21671, 20616]

In [8]:
patent_numbers = sample_df.select('patent_number').distinct()
patent_numbers.show()

+-------------+
|patent_number|
+-------------+
|      5931089|
|      5202128|
|      5997899|
|      5962016|
|      6071534|
|      6066339|
|      5931809|
|      5891467|
|      9072781|
|      6171613|
|      9192608|
|      5807572|
|      6241999|
|      6193998|
|      5378474|
|      5723147|
+-------------+



In [9]:
patent_numbers_list = list(map(lambda x: x.patent_number, patent_numbers.collect()))
patent_numbers_list

['5931089',
 '5202128',
 '5997899',
 '5962016',
 '6071534',
 '6066339',
 '5931809',
 '5891467',
 '9072781',
 '6171613',
 '9192608',
 '5807572',
 '6241999',
 '6193998',
 '5378474',
 '5723147']

## OpenFDA Data (NDA, SPL Set ID)

This will work with the `./spl_id_label_nda.csv` file in this repo.

In [10]:
spls = spark.read.csv('./spl_id_label_nda.csv', header=True, inferSchema=True)
spls.printSchema()

root
 |-- effective_time: integer (nullable = true)
 |-- spl_id: string (nullable = true)
 |-- spl_set_id: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- manufacturer_name: string (nullable = true)
 |-- nda: string (nullable = true)



In [11]:
# Format the NDA application numbers to 'NDAxxxxxx'
appln_numbers_string_list = list(map(lambda x: 'NDA' + str(x).zfill(6), appln_numbers_list))

In [12]:
set_ids = (spls
           .where(col('nda').isin(appln_numbers_string_list))
           .select('spl_set_id')
           .distinct())
set_ids.show(truncate=False)

+------------------------------------+
|spl_set_id                          |
+------------------------------------+
|b62ee1ef-db55-417a-a02e-b103890f9c4b|
|582f42e5-444e-4246-af8c-e7e28097c69a|
|b5cee013-000f-4e35-a284-1f58add31b4d|
|1f53de80-efc8-4930-b3e3-fba0d026af05|
|6fe62c06-5bab-4fc8-b546-ff015a1fc40c|
+------------------------------------+



In [13]:
set_ids_list = list(map(lambda x: x.spl_set_id, set_ids.collect()))
set_ids_list

['b62ee1ef-db55-417a-a02e-b103890f9c4b',
 '582f42e5-444e-4246-af8c-e7e28097c69a',
 'b5cee013-000f-4e35-a284-1f58add31b4d',
 '1f53de80-efc8-4930-b3e3-fba0d026af05',
 '6fe62c06-5bab-4fc8-b546-ff015a1fc40c']

## Patent Data Import

For this, obtain and unzip the USPTO Patent Claim Research dataset from: https://developer.uspto.gov/product/patent-and-patent-application-claims-data-stata-dta-and-ms-excel-csv

Place the unzipped .csv in a folder, accessible to the notebook. The filepath in the notebook is hardcoded to: `data/PatentClaims/patent_claims_fulltext.csv`.

**NOTE**: The unzipped data is about 30GB in size.

In [14]:
patents = spark.read.csv('data/PatentClaims/patent_claims_fulltext.csv', header=True, inferSchema=True)
patents.printSchema()

root
 |-- pat_no: string (nullable = true)
 |-- claim_no: integer (nullable = true)
 |-- claim_txt: string (nullable = true)
 |-- dependencies: string (nullable = true)
 |-- ind_flg: integer (nullable = true)
 |-- appl_id: integer (nullable = true)



In [15]:
selected_patents = patents.where(col('pat_no').isin(patent_numbers_list))
# Cache this to avoid recompute!
selected_patents.cache()

DataFrame[pat_no: string, claim_no: int, claim_txt: string, dependencies: string, ind_flg: int, appl_id: int]

In [16]:
# count all rows (one row per claim)
selected_patents.count()

305

In [17]:
selected_patents.select('pat_no').distinct().count()

14

We have 14 of the 16 patents matched using the Orange Book - the 2 newer ones (9XXXXXX) are not in the research dataset.

In [18]:
selected_patents.show(10)

+-------+--------+--------------------+------------+-------+-------+
| pat_no|claim_no|           claim_txt|dependencies|ind_flg|appl_id|
+-------+--------+--------------------+------------+-------+-------+
|5202128|      26|26. A method for ...|        null|      1|7574551|
|5202128|       1|1. A pH-dependent...|        null|      1|7574551|
|5202128|      19|19. The method of...|          18|      0|7574551|
|5202128|       5|5. The sustained ...|           1|      0|7574551|
|5202128|      20|20. The method of...|          19|      0|7574551|
|5202128|      11|11. The sustained...|          10|      0|7574551|
|5202128|      25|25. A pH-dependen...|        null|      1|7574551|
|5202128|      16|16. The sustained...|          15|      0|7574551|
|5202128|       7|7. The sustained ...|           2|      0|7574551|
|5202128|      15|15. The sustained...|          10|      0|7574551|
+-------+--------+--------------------+------------+-------+-------+
only showing top 10 rows



There is no date info or the Orange Book application number. So, using application number and patent expiration from the Orange Book data instead.

Since there can be more than 1 NDA number for a patent, it will be collected as a list.

In [19]:
ndaPaddingUDF = udf(lambda x: 'NDA' + str(x).zfill(6),StringType())
orange_book_dates = (orange_book_df.select('patent_number', 'patent_expiration', 'application_number')
                     .distinct()
                     .withColumn('application_number', ndaPaddingUDF(col('application_number'))))
orange_book_dates.show()

+-------------+-------------------+------------------+
|patent_number|  patent_expiration|application_number|
+-------------+-------------------+------------------+
|      3705233|1989-12-05 00:00:00|         NDA017559|
|      3432489|1986-03-11 00:00:00|         NDA018333|
|      3957982|1993-05-18 00:00:00|         NDA019192|
|      4952586|2007-08-28 00:00:00|         NDA019677|
|      4018895|1994-04-19 00:00:00|         NDA020101|
|      4056635|1997-03-19 00:00:00|         NDA019627|
|      4507305|2001-05-21 00:00:00|         NDA019661|
|      4188390|2000-10-18 00:00:00|         NDA019668|
|      4724232|2005-09-17 00:00:00|         NDA019951|
|      5057616|2008-10-15 00:00:00|         NDA020529|
|      5116863|2009-05-26 00:00:00|         NDA020688|
|      5004758|2010-05-28 00:00:00|         NDA020671|
|      5900423|2015-05-19 00:00:00|         NDA020885|
|      5514698|2014-03-21 00:00:00|         NDA020968|
|      6063811|2017-05-16 00:00:00|         NDA021052|
|      516

In [20]:
joined_df = (selected_patents
             .join(orange_book_dates, orange_book_dates.patent_number==selected_patents.pat_no)
             .groupBy('pat_no', 'claim_no', 'claim_txt', 'dependencies', 'patent_expiration')
             .agg(collect_set('application_number').alias('application_numbers'))
             .orderBy('pat_no', 'claim_no'))
joined_df.show(3)

+-------+--------+--------------------+------------+-------------------+--------------------+
| pat_no|claim_no|           claim_txt|dependencies|  patent_expiration| application_numbers|
+-------+--------+--------------------+------------+-------------------+--------------------+
|5202128|       1|1. A pH-dependent...|        null|2010-04-13 00:00:00|[NDA022321, NDA02...|
|5202128|       2|2. The sustained ...|           1|2010-04-13 00:00:00|[NDA022321, NDA02...|
|5202128|       3|3. The sustained ...|          35|2010-04-13 00:00:00|[NDA022321, NDA02...|
+-------+--------+--------------------+------------+-------------------+--------------------+
only showing top 3 rows



In [21]:
# Convert to in-memory object
patent_claim_list = joined_df.collect()

In [22]:
# Clear the cached data
selected_patents.unpersist()

DataFrame[pat_no: string, claim_no: int, claim_txt: string, dependencies: string, ind_flg: int, appl_id: int]

### Import Patents to MongoDB

If both MongoDB and Jupyter Notebooks are being run from Docker container, start one service on the network of the other. For example, the below config can be added to the `docker-compose` set up for MongoDB:

```
networks:
  default:
    external:
      name: ds_notebook_default
```

In [23]:
from collections import defaultdict
patent_id_dict = defaultdict(list)
patent_expiration_dict, patent_NDA_dict = {}, {}

# Claims are already ordered in the patent_claim_list
for row in patent_claim_list:
    patent_id_dict[row.pat_no].append({
        'claim_number': row.claim_no,
        'claim_text': row.claim_txt,
        'dependencies': row.dependencies
    })
    patent_expiration_dict[row.pat_no] = row.patent_expiration
    patent_NDA_dict[row.pat_no] = row.application_numbers

In [24]:
# Connect to MongoDB and save data
import pymongo
mongo = pymongo.MongoClient('mongodb://mongo_local:27017')
db = mongo['mockdata']
coll = db['patents']

for patent_id in patent_id_dict:
    # One record per patent
    exp_date = patent_expiration_dict[patent_id]
    record = {
        'patent_number': patent_id,
        'expiration_date': str(exp_date)[:10],
        'published_date': str(exp_date.replace(year=exp_date.year - 10))[:10], # Add mock published date
        'application_numbers': patent_NDA_dict[patent_id],
        'claims': patent_id_dict[patent_id]
    }
    # Upsert into Mongo
    coll.update({'patent_number': patent_id}, record, upsert=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
